## Predição do preço de ações da Petrobrás utilizando RNA com Keras

Aluno: Yann Fabricio Cardoso de Figueiredo <br />
Matrícula: 201704940026

## Descrição da base de dados

O dataset utilizado para fazer a predição proposta é referente ao preço de ações da Petrobrás na bolsa de valores, e este foi retirado do repositório do Yahoo Finanças. Os dados foram extraídos e catalogados de acordo com o valor de abertura e fechamento do dia.
No total há 1267 dados, sendo referentes ao valores de ações dos anos 2013 a 2018.

## Algoritmo

O algoritmo utilizado foi desenvolvido utilizando a biblioteca Keras juntamente de outros frameworks para auxiliar em diversas partes do código como pré-pocessamento de dados, por exemplo. Nas simulações foram utlizados duas versões do algoritmo, sendo um de rede direta e outro recorrente (modelo LSTM). <br />
No geral, como padrão, os algoritmos usam como função de perda o erro quadrático médio, uma porcentagem dropout de 20%, função de ativação linear e o otimizador adam. São utlizadas 4 camadas, sendo a de entrada e ocultas criadas em modelo LSTM e a de saída como Dense.

In [ ]:
#RNA DE REDE RECORRENTE LSTM

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from sklearn.preprocessing import MinMaxScaler

#Leitura da base de dados
base = pd.read_csv("dataset/petr4-treinamento.csv")
base = base.dropna() #Remove as linhas onde há colunas com valores faltantes

#Normalização de dimensionamento de recursos
base_treinamento = base.iloc[:, 1:2].values
normalizador = MinMaxScaler(feature_range=(0, 1))
base_treinamento_normalizada = normalizador.fit_transform(base_treinamento)

#Criando uma estrutura de dados com 90 timesteps e 1 output
x_treino = []
y_treino = []

for i in range(90, 1242):
	x_treino.append(base_treinamento_normalizada[i-90:i, 0])
	y_treino.append(base_treinamento_normalizada[i, 0])

x_treino, y_treino = np.array(x_treino), np.array(y_treino)

#Remodelando
x_treino = np.reshape(x_treino, (x_treino.shape[0], x_treino.shape[1], 1))

### Predição na base de teste ###

base_teste = pd.read_csv('dataset/petr4-teste.csv')
preco_real_teste = base_teste.iloc[:, 1:2].values

base_completa = pd.concat((base['Open'], base_teste['Open']), axis=0)

entradas = base_completa[len(base_completa) - len(base_teste)-90:].values
entradas = entradas.reshape(-1, 1)
entradas = normalizador.transform(entradas)

x_teste = []
for i in range(90, 112):
    x_teste.append(entradas[i-90:i, 0])

x_teste = np.array(x_teste)
x_teste = np.reshape(x_teste, (x_teste.shape[0], x_teste.shape[1], 1))

#Criando modelo
model = Sequential()

#Adicionando camadas LSTM e alguma regularização de Dropout
model.add(LSTM(units=10,return_sequences=True,input_shape=(x_treino.shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(units=10,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=10))
model.add(Dropout(0.2))
model.add(Dense(units=1, activation='linear'))
model.compile(optimizer='adam',loss='mean_squared_error', metrics=['mean_absolute_error'])
model.fit(x_treino, y_treino, epochs=100, batch_size=32)

preco_predito = model.predict(x_teste)
preco_predito = normalizador.inverse_transform(preco_predito)

#Plotando resultados
plt.plot(preco_real_teste, color = 'black', label = 'Preço real')
plt.plot(preco_predito, color = 'green', label = 'Preço predito')
plt.title('Predição do preço de ações da Petrobrás')
plt.xlabel('Tempo')
plt.ylabel('Preço das ações')
plt.legend()
plt.show()

Using TensorFlow backend.


Epoch 1/100
1152/1152 [==============================] - 4s 3ms/step - loss: 0.0753 - mean_absolute_error: 0.2153
Epoch 2/100
 736/1152 [==================>...........] - ETA: 0s - loss: 0.0201 - mean_absolute_error: 0.1088

## Simulações

![title](img/simulacao1.png)
**<div style="text-align: center">Figura 1 - Gráfico obtido na 1ª simulação</div>**

A simulação mostrada na figura 1 proporcionou um modelo de RNA com erro médio quadrático de 0.0183, e é possível observar que o preço predito não condiz ao real com relação a oscilações das ações, o que acaba dificultando a previsão de altas e baixas das ações na bolsa de valores. <br />

A configuração usada foi:
- batch_size = 32
- epochs = 100